In [2]:
import math 
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union
import torch 



from diffusers.configuration_utils import ConfigMixin, register_to_config
from diffusers.utils import BaseOutput
from diffusers.utils.torch_utils import randn_tensor
from diffusers.schedulers.scheduling_utils import SchedulerMixin

## 0.1 DDPM Scheduler output

- DDPM -> Denoising Diffusion Probabilistic Models. 
<p style="color:red">These models are a type of **generative model** used in machine learning, particularly for tasks like **image generation**.
They work by gradually **denoising** a sample from a sample distrubution to match the data distribution, making them effective for generating high-quality image</p>



- `@dataclass`:  The decorator is used to autimatically generate special method like `__init__()` and `__repr__()` for the class, making it easier to create data containers.

* `__repr__()` -> this is the meaning of representation in python. This method is used to define the string representation of an object 

In [5]:
class Representation:

    def __init__(self, name:str):
        self.name = name 


    def __repr__(self):
        return self.name 
    


Representation(name="hello")

hello

In [9]:
@dataclass
class DDPMSchedulerOutput(BaseOutput):

    """ 
    Output class for the scheduler step function output.

    Args:
        prev_sample (`torch.Tensor` of shape `(batch_size, num_channels, height, width`) for images):
        Computed sample (x_{t-1}) of previous timestep. `prev_sample` should be used as next model input in the 
        denoising loop.

    """

    prev_sample: torch.Tensor



prev_sample = torch.Tensor([2])
# print(prev_sample)
DDPMSchedulerOutput(prev_sample)

tensor([2.])


DDPMSchedulerOutput(prev_sample=tensor([2.]))

## 0.2 DDPMS Cosine Scheduler 

- `register_to_config`: This decorator register the configuration parameters.
- `__init__()` method initilizes the scheduler with `scalar` and `s` **parameter** , Computes `_init_alpha_cumprod` and set the `init_noise_sigma`

- `_init_alpha_cumprod` method computes the cumulative product of alpha values, adjusting `t` based on the scaler and using cosine function to calculate the values.

--------------





In [38]:
class DDPMCosineScheduler(SchedulerMixin, ConfigMixin):

    @register_to_config
    def __init__(self,
                scaler: float = 1.0,
                s: float = 0.008
                ):
        
        super().__init__()
        self.s = torch.tensor([s])
        self._init_alpha_cumprod = torch.cos(self.s / (1 + self.s) * torch.pi * 0.5) ** 2



        # standard deviation of the initial noise distribution 
        self.init_noise_sigma = 1.0 


    def _alpha_cumprod(self, t, device):

        if self.scaler > 1: 
            t = 1 - (1 - t) ** self.scaler 



        elif self.scaler < 1:
            t = t ** self.scaler 


        alpha_cumprod = torch.cos(
            (t + self.s.to(device)) / (1 + self.s.to(device)) * torch.pi * 0.5 
        ) ** 2 / self._init_alpha_cumprod.to(device)


        return alpha_cumprod.clamp(0.0001, 0.9999)
    


    def scale_model_input(self,
                          sample: torch.Tensor,
                          timestep: Optional[int] = None
                          ) -> torch.Tensor:
        

        """ 
        Ensures interchangeability with schedulers that need to scale the denoising model input depending on the 
        current timestep.


        Args: 
            sample (`torch.Tensor`): input sample 
            timestep (`init`, optional): current timestep 


        Returns:
            `torch.Tensor`: scaled input sample
        
        """

        return sample
    


    def set_timestep(
            self,
            num_inference_steps: int = None,
            timesteps: Optional[List[int]] = None, 
            device: Union[str, torch.device] = None,
    ):
        

        """ 
        Sets the discreate timesteps used for the diffusion chain. Supporting function to be run before inference.

        Args:
            num_inference_steps (`Dict[float, int]`):
                the number of diffusion steps used when generating samples with a pre-trained model. If passed, then 
                `timesteps` must be `None`. 

            device (`str` or `torch.device`, optional):
                the device to which the timesteps are moved to. {2 / 3: 20, 0.0: 10}
        """


        if timesteps is None:
            timesteps = torch.linspace(1.0, 0.0, num_inference_steps + 1, device)

        
        if not isinstance(timesteps, torch.Tensor):
            timesteps = torch.Tensor(timesteps).to(device)


        self.timesteps = timesteps



    # This function is part of the denoising process in a diffusion model, where it computes the next sample in the sequence
    # based on the current model output and timestep.
    def step(
            self,
            model_output: torch.Tensor,                 # the output tensor from the model.
            timestep: int,                              # the current timestep in the denoising process
            sample: torch.Tensor,                       # the current sample tensor
            generator: None,                            # A random number generator (optional)
            return_dict: bool = True                    # A boolean indicating whether to return a dictionary or a tuple
    ) -> Union[DDPMSchedulerOutput, Tuple]:
        

        # initilize variables 
        dtype = model_output.dtype
        device = model_output.device
        t = timestep

        prev_t = self.previous_timestep(t)

        # Compute Alpha Cumulative Products
        alpha_cumprod = self._alpha_cumprod(t, device).view(t.size(0), *[1 for _ in sample.shape[1:]])
        alpha_cumprod_prev = self._alpha_cumprod(prev_t, device).view(prev_t.size(0), *[1 for _ in sample.shape[1:]])
        alpha = alpha_cumprod / alpha_cumprod_prev

        # compute mean
        mu = (1.0 / alpha).sqrt() * (sample - (1 - alpha) * model_output / (1 - alpha_cumprod).sqrt())


        # Generate Standard Noise and Compute standard Deviation 
        std_noise = randn_tensor(mu.shape, generator=generator, device=model_output.device, dtype=model_output.dtype)
        std = ((1 - alpha) * (1.0 - alpha_cumprod_prev) / (1.0 - alpha_cumprod)).sqrt() * std_noise


        # compute preduction 
        pred = mu + std * (prev_t != 0).float().view(prev_t.size(0), *[1 for _ in sample.shape[1:]])


        # return results
        if not return_dict:
            return (pred.to(dtype),)


        return DDPMSchedulerOutput(prev_sample=pred.to(dtype))
    


    # This functin is typically used in diffusion model to add noise to the amples at each timestep, simulating the forward 
    # diffusion process.
    def add_noise(
            self, 
            original_samples: torch.Tensor,       # the original tensor samples to which noise will be added
            noise: torch.Tensor,                  # the noise tensor to be added to the original samples
            timesteps: torch.Tensor               # the timesteps tensor indicating the current step in the diffusion process

    ) -> torch.Tensor:
        

        # initialize variables
        device = original_samples.device
        dtype = original_samples.dtype



        # compute alpha comulative products
        alpha_cumprod = self._alpha_cumprod(timesteps, device=device).view(
            timesteps.size(0), *[1 for _ in original_samples.shape[1:]]
        )

        # compute noise samples
        noisy_samples = alpha_cumprod.sqrt() * original_samples + (1 - alpha_cumprod).sqrt() * noise 

        # return noise samples
        return noisy_samples.to(dtype=dtype)
    


    def __len__(self):
        return self.config.num_train_tiemsteps 
    



    def previous_timestep(self, timestep):
        index = (self.timesteps - timestep[0]).abs().argmin().item()
        prev_t = self.timesteps[index + 1][None].extend(timestep.shape[0])
        return prev_t
    



    

    
    


    




    




## Example

### 0.1 _alpha_cumprod

In [54]:
class DDPMCosineScheduler(SchedulerMixin, ConfigMixin):

    @register_to_config
    def __init__(self,
                scaler: float = 1.0,
                s: float = 0.008
                ):
        
        super().__init__()
        self.s = torch.tensor([s])
        self._init_alpha_cumprod = torch.cos(self.s / (1 + self.s) * torch.pi * 0.5) ** 2



        # standard deviation of the initial noise distribution 
        self.init_noise_sigma = 1.0 


    def _alpha_cumprod(self, t, device):

        if self.scaler > 1: 
            t = 1 - (1 - t) ** self.scaler 
            print("if is working")



        elif self.scaler < 1:
            t = t ** self.scaler 
            print("elif is working")


        alpha_cumprod = torch.cos(
            (t + self.s.to(device)) / (1 + self.s.to(device)) * torch.pi * 0.5 
        ) ** 2 / self._init_alpha_cumprod.to(device)


        return alpha_cumprod.clamp(0.0001, 0.9999)
    

# Example
scheduler = DDPMCosineScheduler(scaler=1.2, s=0.01)
t = torch.tensor([0.5])
device = torch.device('cpu')
alpha_cumprod = scheduler._alpha_cumprod(t, device)
print(alpha_cumprod)


if is working
tensor([0.3925])


c:\Users\Manjusha Kumari\AppData\Local\Programs\Python\Python312\Lib\site-packages\diffusers\configuration_utils.py:140: FutureWarning: Accessing config attribute `scaler` directly via 'DDPMCosineScheduler' object attribute is deprecated. Please access 'scaler' over 'DDPMCosineScheduler's config object instead, e.g. 'scheduler.config.scaler'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


In [60]:
class DDPMCosineScheduler(SchedulerMixin, ConfigMixin):

    @register_to_config
    def __init__(self,
                scaler: float = 1.0,
                s: float = 0.008
                ):
        
        super().__init__()
        self.s = torch.tensor([s])
        self._init_alpha_cumprod = torch.cos(self.s / (1 + self.s) * torch.pi * 0.5) ** 2



        # standard deviation of the initial noise distribution 
        self.init_noise_sigma = 1.0 


    


    def set_timesteps(
        self,
        num_inference_steps: int = None,
        timesteps: Optional[List[int]] = None,
        device: Union[str, torch.device] = None,
    ):
        """
        Sets the discrete timesteps used for the diffusion chain. Supporting function to be run before inference.

        Args:
            num_inference_steps (`Dict[float, int]`):
                the number of diffusion steps used when generating samples with a pre-trained model. If passed, then
                `timesteps` must be `None`.
            device (`str` or `torch.device`, optional):
                the device to which the timesteps are moved to. {2 / 3: 20, 0.0: 10}
        """
        if timesteps is None:
            timesteps = torch.linspace(1.0, 0.0, num_inference_steps + 1, device=device)
        if not isinstance(timesteps, torch.Tensor):
            timesteps = torch.Tensor(timesteps).to(device)
        self.timesteps = timesteps





# Example usage 
scheduler = DDPMCosineScheduler()
num_inference_steps = 1000 
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

scheduler.set_timesteps(num_inference_steps=num_inference_steps,
                        device=device
                        )
print(scheduler.timesteps)


# set timesteps using a custom list 
custom_timesteps = [0.0, 0.5, 1.0]
scheduler.set_timesteps(timesteps=custom_timesteps, device=device)
print(scheduler.timesteps)


tensor([1.0000, 0.9990, 0.9980,  ..., 0.0020, 0.0010, 0.0000], device='cuda:0')
tensor([0.0000, 0.5000, 1.0000], device='cuda:0')
